<h1>First section of the project</h1>
    This section contains the first part of the project, where we are required to scrape the dtaset from the given wikipedia page

In [116]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(page, 'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"Xg1m4ApAICkAALDKtD8AAABJ","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":933624196,"wgRevisionId":933624196,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communi

In [3]:
MyTable = soup.find('table',{'class':'wikitable sortable'})
MyTable

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_

In [4]:
ths = MyTable.find_all('th')
headings = [th.text.strip() for th in ths]
headings

['Postcode', 'Borough', 'Neighborhood']

In [5]:
rows = MyTable.find_all('tr')
data = []
for row in rows:
    data.append([t.text.strip() for t in row.find_all('td')])

In [15]:
df = pd.DataFrame(data, columns = headings)
df_Postcodes = df[df.Borough != 'Not assigned'].reset_index(drop=True)
df_Postcodes = df_Postcodes.iloc[1:].reset_index(drop=True)
df_Postcodes.head(15)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Not assigned
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


<h1>Second section</h1>
This section contains the second section of the project where we are required to get the lattitudes and logitudes using the postal data

In [26]:
lonlatData = pd.read_csv('http://cocl.us/Geospatial_data')
lonlatData.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [31]:
dfinal = df_Postcodes.merge(lonlatData, how='inner', left_on='Postcode', right_on='Postal Code')
dfinal.head()

,Postcode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,M5A,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,M6A,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,M6A,43.718518,-79.464763


<h1>Third section</h1>
In this section we are required to do the cluster analysis based on the datset developed in earlier sections.

In [37]:
# Get the lon-lat for Toronto
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude_tr = location.latitude
longitude_tr = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_tr, longitude_tr))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [42]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude_tr, longitude_tr], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfinal['Latitude'], dfinal['Longitude'], dfinal['Borough'], dfinal['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [44]:
# Foursquare credentials
CLIENT_ID = 'GYQ1PPNIN4JOPSJDUZ00QPW12YGINODNK2QW4LT1DDGLYUTK' # your Foursquare ID
CLIENT_SECRET = 'VRVXQAY3ICMA3YUUS3PDTSWQCZUEZMZJYE2MRGFDZE1LH4DX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GYQ1PPNIN4JOPSJDUZ00QPW12YGINODNK2QW4LT1DDGLYUTK
CLIENT_SECRET:VRVXQAY3ICMA3YUUS3PDTSWQCZUEZMZJYE2MRGFDZE1LH4DX


In [45]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [56]:
# type your answer here
LIMIT = 100
toronto_venues = getNearbyVenues(names=dfinal['Neighborhood'],
                                   latitudes=dfinal['Latitude'],
                                   longitudes=dfinal['Longitude']
                                  )


Parkwoods
Victoria Village
Harbourfront
Lawrence Heights
Lawrence Manor
Queen's Park
Not assigned
Rouge
Malvern
Don Mills North
Woodbine Gardens
Parkview Hill
Ryerson
Garden District
Glencairn
Cloverdale
Islington
Martin Grove
Princess Gardens
West Deane Park
Highland Creek
Rouge Hill
Port Union
Flemingdon Park
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens
Eringate
Markland Wood
Old Burnhamthorpe
Guildwood
Morningside
West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor
Downsview North
Wilson Heights
Thorncliffe Park
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Scarborough Village
Fairview
Henry Farm
Oriole
Northwood Park
York University
East Toronto
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
East Birchmount Park
Ionview
Kennedy Park
Bayview Village
CFB Toronto
Downsview East
The Danforth West
Riverdale
Design Exchange
Toronto 

In [57]:
print(toronto_venues.shape)
toronto_venues.head()

(4333, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [58]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
Agincourt North,3,3,3,3,3,3
Albion Gardens,10,10,10,10,10,10
Alderwood,9,9,9,9,9,9
Bathurst Manor,19,19,19,19,19,19
Bathurst Quay,17,17,17,17,17,17
Bayview Village,4,4,4,4,4,4
Beaumond Heights,10,10,10,10,10,10


In [59]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 270 uniques categories.


In [106]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot.drop(['Neighborhood'],axis = 1)

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [107]:
toronto_onehot_.shape

(4333, 271)

In [109]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Adelaide,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,...,0.00,0.020000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.01
1,Agincourt,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,Agincourt North,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,Albion Gardens,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,Alderwood,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
5,Bathurst Manor,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Bathurst Quay,0.000000,0.0,0.058824,0.058824,0.058824,0.117647,0.117647,0.117647,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
7,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
8,Beaumond Heights,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,Bedford Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00


In [111]:
toronto_grouped.shape

(199, 270)

In [113]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
         venue  freq
0  Coffee Shop  0.08
1         Café  0.04
2   Steakhouse  0.04
3          Bar  0.04
4        Hotel  0.03


----Agincourt----
                       venue  freq
0                     Lounge  0.25
1             Breakfast Spot  0.25
2  Latin American Restaurant  0.25
3               Skating Rink  0.25
4              Metro Station  0.00


----Agincourt North----
           venue  freq
0     Playground  0.33
1         Bakery  0.33
2           Park  0.33
3    Yoga Studio  0.00
4  Metro Station  0.00


----Albion Gardens----
                  venue  freq
0           Pizza Place   0.2
1         Grocery Store   0.2
2           Video Store   0.1
3        Sandwich Place   0.1
4  Fast Food Restaurant   0.1


----Alderwood----
         venue  freq
0  Pizza Place  0.22
1  Coffee Shop  0.11
2          Pub  0.11
3     Pharmacy  0.11
4          Gym  0.11


----Bathurst Manor----
                  venue  freq
0           Coffee Shop  0.11
1           Gas Station  0.

                             venue  freq
0                          Airport   0.5
1                             Park   0.5
2                      Yoga Studio   0.0
3                    Metro Station   0.0
4  Molecular Gastronomy Restaurant   0.0


----Downsview North----
                  venue  freq
0           Coffee Shop  0.11
1           Gas Station  0.05
2                 Diner  0.05
3         Shopping Mall  0.05
4  Fast Food Restaurant  0.05


----Downsview Northwest----
                  venue  freq
0          Liquor Store   0.2
1        Discount Store   0.2
2  Gym / Fitness Center   0.2
3                   Gym   0.2
4         Grocery Store   0.2


----Downsview West----
           venue  freq
0  Grocery Store  0.33
1  Shopping Mall  0.17
2           Bank  0.17
3           Park  0.17
4          Hotel  0.17


----Dufferin----
         venue  freq
0       Bakery  0.11
1     Pharmacy  0.11
2  Supermarket  0.11
3  Art Gallery  0.06
4      Brewery  0.06


----East Birchmount Park----

                             venue  freq
0                   Baseball Field   1.0
1                      Yoga Studio   0.0
2               Mexican Restaurant   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0


----Kingsview Village----
                       venue  freq
0                Pizza Place  0.25
1             Sandwich Place  0.25
2          Mobile Phone Shop  0.25
3                   Bus Line  0.25
4  Middle Eastern Restaurant  0.00


----Kingsway Park South East----
                             venue  freq
0                   Baseball Field   1.0
1                      Yoga Studio   0.0
2               Mexican Restaurant   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0


----Kingsway Park South West----
            venue  freq
0          Bakery  0.08
1  Sandwich Place  0.08
2     Wings Joint  0.08
3    Burger Joint  0.08
4  Hardware Store  0.08


----L'Amoreaux East----
           venue  freq
0     P

                venue  freq
0         Coffee Shop  0.28
1    Sushi Restaurant  0.05
2                 Gym  0.05
3                Park  0.05
4  College Auditorium  0.03


----Railway Lands----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3       Coffee Shop  0.06
4           Airport  0.06


----Rathnelly----
                venue  freq
0         Coffee Shop  0.13
1                 Pub  0.13
2  Light Rail Station  0.07
3        Liquor Store  0.07
4          Sports Bar  0.07


----Richmond----
         venue  freq
0  Coffee Shop  0.08
1         Café  0.04
2   Steakhouse  0.04
3          Bar  0.04
4        Hotel  0.03


----Richview Gardens----
                       venue  freq
0                Pizza Place  0.25
1             Sandwich Place  0.25
2          Mobile Phone Shop  0.25
3                   Bus Line  0.25
4  Middle Eastern Restaurant  0.00


----Riverdale----
                venue  freq
0    Greek Restaurant  0.21
1     

                venue  freq
0                Café  0.09
1     Thai Restaurant  0.09
2  Mexican Restaurant  0.09
3         Music Venue  0.05
4           Gastropub  0.05


----The Kingsway----
           venue  freq
0           Park  0.33
1          River  0.33
2           Pool  0.33
3    Yoga Studio  0.00
4  Metro Station  0.00


----The Queensway East----
                             venue  freq
0                   Baseball Field   1.0
1                      Yoga Studio   0.0
2               Mexican Restaurant   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0


----The Queensway West----
            venue  freq
0          Bakery  0.08
1  Sandwich Place  0.08
2     Wings Joint  0.08
3    Burger Joint  0.08
4  Hardware Store  0.08


----Thistletown----
                  venue  freq
0           Pizza Place   0.2
1         Grocery Store   0.2
2           Video Store   0.1
3        Sandwich Place   0.1
4  Fast Food Restaurant   0.1


----Thorncliffe Par

In [114]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [138]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,Restaurant,Burger Joint,Hotel,Sushi Restaurant,Asian Restaurant
1,Agincourt,Latin American Restaurant,Lounge,Skating Rink,Breakfast Spot,Women's Store,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,Agincourt North,Park,Bakery,Playground,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
3,Albion Gardens,Grocery Store,Pizza Place,Fried Chicken Joint,Pharmacy,Video Store,Fast Food Restaurant,Beer Store,Sandwich Place,Women's Store,Dog Run
4,Alderwood,Pizza Place,Coffee Shop,Pool,Gym,Skating Rink,Pharmacy,Pub,Sandwich Place,Dessert Shop,Dim Sum Restaurant


In [139]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 2, 1, 1, 1, 1, 1, 1, 1])

In [140]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = dfinal

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656,2.0,Park,Food & Drink Shop,Women's Store,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572,1.0,Portuguese Restaurant,Pizza Place,Intersection,Coffee Shop,Hockey Arena,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant
2,M5A,Downtown Toronto,Harbourfront,M5A,43.654260,-79.360636,1.0,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Mexican Restaurant,Brewery,Shoe Store,Restaurant
3,M6A,North York,Lawrence Heights,M6A,43.718518,-79.464763,1.0,Clothing Store,Accessories Store,Furniture / Home Store,Women's Store,Coffee Shop,Miscellaneous Shop,Arts & Crafts Store,Boutique,Vietnamese Restaurant,Cosmetics Shop
4,M6A,North York,Lawrence Manor,M6A,43.718518,-79.464763,1.0,Clothing Store,Accessories Store,Furniture / Home Store,Women's Store,Coffee Shop,Miscellaneous Shop,Arts & Crafts Store,Boutique,Vietnamese Restaurant,Cosmetics Shop


In [141]:
toronto_merged = toronto_merged.dropna(subset=['Cluster Labels'])
print(toronto_merged[toronto_merged['Cluster Labels'].isnull()])

Empty DataFrame
Columns: [Postcode, Borough, Neighborhood, Postal Code, Latitude, Longitude, Cluster Labels, 1st Most Common Venue, 2nd Most Common Venue, 3rd Most Common Venue, 4th Most Common Venue, 5th Most Common Venue, 6th Most Common Venue, 7th Most Common Venue, 8th Most Common Venue, 9th Most Common Venue, 10th Most Common Venue]
Index: []


In [142]:

# create map
map_clusters = folium.Map(location=[latitude_tr, longitude_tr], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters